# Agents

Agent is another hot topic for Gen AIs.

Still taking the Bedrock question as example, what if we offer LLMs some `tools` to use? such as LLM can search from internet for information if it doesn't know.

LangChain has support of Agents and tools, and it's more than just search from internet.  

<img src="assets/agent.png" width="1000" />

(Image from LangChain doc)

In this notebook, we will use some Agents and Tools in LangChain, you can try and test others by referring the LangChain documents.

So far, the open source language models don't perform well in this area, we will have to use OpenAI, make sure you have an OpenAI key if you want to try this notebook by yourself.

The LangChain version I have tested in this notebook is v0.0.273.

In [ ]:
!pip install openai

In [1]:
from langchain.agents import Tool, AgentType, initialize_agent

In [2]:
import os

# Change to use your own key
os.environ["OPENAI_API_KEY"] = "sk-xxx"

In [ ]:
!pip install openai

In [3]:
from langchain import OpenAI
llm=OpenAI(model="text-davinci-003", temperature=0.0)

In [6]:
llm.predict("Hello")

',\n\nI am writing to you to inquire about the availability of a room in your hotel. I am planning to stay in your hotel for a few days and would like to know if you have any rooms available.\n\nI am looking for a room with a double bed, a private bathroom, and a balcony. I would also like to know if there are any additional amenities such as a mini-fridge, a microwave, or a coffee maker.\n\nI look forward to hearing from you soon.\n\nSincerely,\n[Your Name]'

We use use two tools in this example to show you how LLMs can decide what tools to use.

For using SerpAPIWrapper, you will need to have API key.

In [ ]:
!pip install google-search-results

In [4]:
# Change to use your own SerpAPI key
os.environ["SERPAPI_API_KEY"] = "ee66..."

In [5]:
from langchain import LLMMathChain, SerpAPIWrapper

search = SerpAPIWrapper(search_engine="bing")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="A search engine. Useful for when you need to answer questions about current events. Input should be a search query."
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions about math."
    ),
]

In [6]:
tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='ee66907d30dc939838dfb85b80daaba246fa345f88931d69bd8203aa4ce9faac', aiosession=None)>, coroutine=None),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, llm_ch

If you want to see more details of how the agents work, you can also turn on the debug mode

## Basic Agent

ReAct: Reasoning and Acting

In [7]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

conv_agent = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory, verbose=True)

In [10]:
conv_agent.run(input="hello")



> Entering new AgentExecutor chain...



Thought: Do I need to use a tool? No
AI: Hi there! How can I help you?

> Finished chain.


'Hi there! How can I help you?'

In [11]:
conv_agent.run(input="what is 1+1")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 1+1

> Entering new LLMMathChain chain...
1+1

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


```text
1+1
```
...numexpr.evaluate("1+1")...

Answer: 2
> Finished chain.

Observation: Answer: 2
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 Do I need to use a tool? No
AI: The answer is 2.

> Finished chain.


'The answer is 2.'

In [11]:
conv_agent.run(input="What language models are supported by Amazon Bedrock?")



> Entering new AgentExecutor chain...



Thought: Do I need to use a tool? No
AI: Amazon Bedrock supports a variety of language models, including natural language processing (NLP), machine learning (ML), and deep learning (DL). It also provides a range of tools and services to help developers quickly and easily build, deploy, and manage applications on the AWS cloud.

> Finished chain.


'Amazon Bedrock supports a variety of language models, including natural language processing (NLP), machine learning (ML), and deep learning (DL). It also provides a range of tools and services to help developers quickly and easily build, deploy, and manage applications on the AWS cloud.'

In [12]:
print(conv_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

Zero Shot ReAct

In [13]:
zero_shot_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [17]:
zero_shot_agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...


 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Blake Lively and DiCaprio are believed to have enjoyed a whirlwind five-month romance in 2011. The pair were seen on a yacht together in Cannes, ...
Thought: I need to find out how old Blake Lively is.
Action: Search
Action Input: "Blake Lively age"
Observation: 36 years
Thought: I need to calculate 36 raised to the 0.43 power.
Action: Calculator
Action Input: 36^0.43

> Entering new LLMMathChain chain...
36^0.43

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

```text
36**0.43
```
...numexpr.evaluate("36**0.43")...

Answer: 4.6688516567750975
> Finished chain.

Observation: Answer: 4.6688516567750975
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer.
Final Answer: Blake Lively, Leo DiCaprio's girlfriend, is 36 years old and her age raised to the 0.43 power is 4.6688516567750975.

> Finished chain.


"Blake Lively, Leo DiCaprio's girlfriend, is 36 years old and her age raised to the 0.43 power is 4.6688516567750975."

One thing you may have noticed is that it will call LLM many times.

In [15]:
zero_shot_agent.run(input="What language models are supported by Amazon Bedrock?")



> Entering new AgentExecutor chain...
 I need to research what language models are supported by Amazon Bedrock.
Action: Search
Action Input: "Amazon Bedrock language models"
Observation: Amazon Bedrock is a fully managed service that makes foundation models (FMs) from Amazon and leading AI startups available through an API, so you can choose ...
Thought: I need to find out which language models are supported by Amazon Bedrock.
Action: Search
Action Input: "Amazon Bedrock supported language models"
Observation: Amazon Bedrock is a fully managed service that makes foundation models (FMs) from Amazon and leading AI startups available through an API, so you can choose ...
Thought: I now know the language models supported by Amazon Bedrock.
Final Answer: Amazon Bedrock supports foundation models (FMs) from Amazon and leading AI startups.

> Finished chain.


'Amazon Bedrock supports foundation models (FMs) from Amazon and leading AI startups.'

In [16]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## Python Agent

What if LLMs can write and execute Python code by itself.

In [15]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool


python_agent = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [29]:
python_agent.run("What is the 10th fibonacci number?")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to calculate the 10th fibonacci number
Action: Python_REPL
Action Input: def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))
Observation: 55

Thought: I now know the final answer
Final Answer: The 10th fibonacci number is 55.

> Finished chain.


'The 10th fibonacci number is 55.'

In [30]:
python_agent.run("Sort a list of numbers 1, 9, 3, 4, 7, 6 and return the result in ascending order")



> Entering new AgentExecutor chain...
 I need to sort the list
Action: Python_REPL
Action Input: my_list = [1, 9, 3, 4, 7, 6]; my_list.sort()
Observation: 
Thought: I now know the sorted list
Final Answer: [1, 3, 4, 6, 7, 9]

> Finished chain.


'[1, 3, 4, 6, 7, 9]'

In [31]:
print(python_agent.agent.llm_chain.prompt.template)

You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python_REPL]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times

## CSV Agent

With CSV agent, you can chat with your csv file. This agent calls the Pandas DataFrame agent under the hood.

In [16]:
!pip install pandas tabulate

In [ ]:
import pandas as pd
data = pd.read_csv("data/titanic_train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
# CSV agent
from langchain.agents import create_csv_agent

csv_agent = create_csv_agent(
    llm,
    "data/titanic_train.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [18]:
csv_agent.run("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people with more than 3 siblings
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].shape[0]
Observation: 30
Thought: I now know the final answer
Final Answer: 30 people have more than 3 siblings.

> Finished chain.


'30 people have more than 3 siblings.'

In [19]:
csv_agent.run("how many people are female?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people who are female
Action: python_repl_ast
Action Input: df[df['Sex'] == 'female'].shape[0]
Observation: 314
Thought: I now know the final answer
Final Answer: 314

> Finished chain.


'314'

In [20]:
print(csv_agent.agent.llm_chain.prompt.template)


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}


## SQL Agent

You can also chat with your database.

In [21]:
# Database
from sqlalchemy import create_engine

# Create your engine.
engine = create_engine("sqlite:///test.db")

In [23]:
data.to_sql('passenger', con=engine)

891

In [24]:
pd.read_sql_query("SELECT count(*) FROM passenger", engine)

,count(*)
0,891


In [25]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

In [26]:
from langchain.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///test.db")
# llm = OpenAI(temperature=1.0, verbose=True)

sql_agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [27]:
# sql_agent.run(
#     #"how many records in total in the table?"
    
# )
sql_agent.run("how many passengers in total")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: passenger
Thought: I should query the schema of the passenger table
Action: sql_db_schema
Action Input: passenger
Observation: 
CREATE TABLE passenger (
	"index" BIGINT, 
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from passenger table:
index	PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
0	1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
1	2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
2	3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


 I should query the total number of passengers
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM passenger
Observation: [(891,)]
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer
Final Answer: There are 891 passengers in total.

> Finished chain.


'There are 891 passengers in total.'

In [28]:
sql_agent.run("how many passengers are female?")



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


Action: sql_db_list_tables
Action Input: 
Observation: passenger
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I should query the schema of the passenger table
Action: sql_db_schema
Action Input: passenger
Observation: 
CREATE TABLE passenger (
	"index" BIGINT, 
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from passenger table:
index	PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
0	1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
1	2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
2	3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I should query the database for the number of female passengers
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM passenger WHERE Sex = 'female';
Observation: [(314,)]
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-LWwSFZw5jX56JXIHA8vc52uu on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer
Final Answer: There are 314 female passengers.

> Finished chain.


'There are 314 female passengers.'